In [17]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  2  955k    2 19883    0     0  20880      0  0:00:46 --:--:--  0:00:46 20907
  9  955k    9 93323    0     0  45087      0  0:00:21  0:00:02  0:00:19 45127
 15  955k   15  151k    0     0  51508      0  0:00:18  0:00:03  0:00:15 51541
 22  955k   22  213k    0     0  55481      0  0:00:17  0:00:03  0:00:14 55497
 30  955k   30  286k    0     0  59380      0  0:00:16  0:00:04  0:00:12 59385
 40  955k   40  387k    0     0  64810      0  0:00:15  0:00:06  0:00:09 72904
 48  955k   48  463k    0     0  67363      0  0:00:14  0:00:07  0:00:07 76611
 56  955k   56  539k    0     0  69549      0  0:00:14  0:00:07  0:00:07 80567
 64  955k   64  615k    0     0  69189      0  0:00:14  0:00:09  0:00:05 79644
 72  955k   72  694k    0     0  71059      0  0:00

In [18]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [19]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [20]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [21]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [22]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [23]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [24]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    # def predict(self, user, item):
    #     return self.forward(user, item)

    def predict(self, data):
        users, items = data[:, 0], data[:, 1]
        return self.forward(data)

    def recommend_movies(self, user_id, n=10):
        """
        Recommends top-N movies for a given user.

        Parameters:
            user_id (int): User ID for whom recommendations are made.
            n (int): Number of recommendations to return.

        Returns:
            list: List of movie IDs recommended for the user.
        """
        # Convert user ID to continuous index
        user_idx = train_set.userid2idx[user_id]
        print(f"user index: {user_idx}" )

        # Get all movie indices
        all_movie_indices = torch.arange(len(train_set.idx2movieid))
        print(f"all movie indices: {all_movie_indices}")

        # Create user-movie pairs for predictions
        user_movie_indices = torch.cartesian_prod(torch.tensor([user_idx]), all_movie_indices)
        print(f"Create user-movie pairs for predictions: {user_movie_indices}")

        # Combine user and movie indices into a single tensor
        user_movie_tensor = torch.tensor(list(zip(user_movie_indices[:, 0], user_movie_indices[:, 1])))
        print(f" Combine user and movie indices into a single tensor: {user_movie_tensor}")

        # Move the input tensor to GPU if available
        if cuda:
            user_movie_tensor = user_movie_tensor.cuda()

        # Make predictions for all user-movie pairs
        predictions = self.predict(user_movie_tensor)

        # Sort movie indices based on predicted ratings in descending order
        sorted_indices = torch.argsort(predictions, descending=True)

        # Extract top-N movie indices
        top_n_indices = sorted_indices[:n]

        # Convert movie indices back to movie IDs
        top_n_movie_ids = [train_set.idx2movieid[idx.item()] for idx in top_n_indices]

        return top_n_movie_ids

In [25]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [26]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0056, 0.0132, 0.0414,  ..., 0.0498, 0.0086, 0.0200],
        [0.0141, 0.0377, 0.0091,  ..., 0.0236, 0.0154, 0.0446],
        [0.0395, 0.0002, 0.0163,  ..., 0.0276, 0.0180, 0.0232],
        ...,
        [0.0258, 0.0215, 0.0195,  ..., 0.0057, 0.0251, 0.0433],
        [0.0494, 0.0229, 0.0222,  ..., 0.0435, 0.0377, 0.0020],
        [0.0034, 0.0370, 0.0248,  ..., 0.0351, 0.0348, 0.0232]])
item_factors.weight tensor([[0.0278, 0.0273, 0.0250,  ..., 0.0408, 0.0346, 0.0437],
        [0.0301, 0.0046, 0.0133,  ..., 0.0239, 0.0320, 0.0325],
        [0.0132, 0.0262, 0.0387,  ..., 0.0116, 0.0418, 0.0119],
        ...,
        [0.0081, 0.0041, 0.0302,  ..., 0.0117, 0.0344, 0.0145],
        [0.0395, 0.0341, 0.0351,  ..., 0.0043, 0.0168, 0.0036],
        [0.0339, 0.0274, 0.0468,  ..., 0.0215, 0.0162, 0.0041]])


In [27]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

C:\Users\Roze\AppData\Local\Temp\ipykernel_8456\59576823.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.1034,  1.4892,  0.7528,  ...,  1.3036,  1.0512,  1.5724],
        [ 1.5081, -0.0813,  0.2775,  ...,  1.9494,  0.9154,  1.0569],
        [ 0.2833,  0.0605,  0.9062,  ...,  2.8402, -1.6074,  1.3509],
        ...,
        [ 2.7284,  0.2840,  0.0313,  ..., -0.0923,  0.6011,  1.0638],
        [ 0.8062,  0.7465,  0.9239,  ...,  1.4485,  1.0231,  1.1420],
        [ 1.1826,  1.3374,  1.4770,  ...,  0.9632,  0.8720,  1.3309]],
       device='cuda:0')
item_factors.weight tensor([[0.4605, 0.5332, 0.8383,  ..., 0.3537, 0.4206, 0.3439],
        [0.2664, 0.2682, 0.4361,  ..., 1.0213, 0.2412, 0.4369],
        [0.6715, 0.5397, 0.6165,  ..., 0.6072, 0.4597, 0.0573],
        ...,
        [0.3439, 0.3727, 0.3536,  ..., 0.3634, 0.3803, 0.3565],
        [0.4024, 0.3834, 0.3954,  ..., 0.4149, 0.3882, 0.3858],
        [0.4275, 0.4095, 0.4092,  ..., 0.3881, 0.4205, 0.4229]],
       device='cuda:0')


In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Ace Ventura: Pet Detective (1994)
	 Mask, The (1994)
	 Pirates of the Caribbean: The Curse of the Black Pearl (2003)
	 Dumb & Dumber (Dumb and Dumber) (1994)
	 Clueless (1995)
	 Happy Gilmore (1996)
	 While You Were Sleeping (1995)
	 Ace Ventura: When Nature Calls (1995)
	 Mummy, The (1999)
	 Jerry Maguire (1996)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Braveheart (1995)
	 Toy Story (1995)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #2
	 Pulp Fiction (1994)
	 Fight Club (1999)
	 Usual Suspects, The (1995)
	 American Beauty (1999)
	 Godfather, The (1972)
	 Fargo (1996)
	 Memento (2000)
	 Alien (1979)
	 Inception (2010)
	 Monty Python and the Holy Grail (1975)
Cluster #3
	 Jurassic 

In [ ]:
# Assuming 'user_id' is the user for whom recommendations are requested
user_id = 3

# Make recommendations for the user
recommended_movies = model.recommend_movies(user_id, n=10)

# Print the recommended movies
print("Recommended Movies for User {}: {}".format(user_id, recommended_movies))

user index: 2
all movie indices: tensor([   0,    1,    2,  ..., 9721, 9722, 9723])
Create user-movie pairs for predictions: tensor([[   2,    0],
        [   2,    1],
        [   2,    2],
        ...,
        [   2, 9721],
        [   2, 9722],
        [   2, 9723]])
 Combine user and movie indices into a single tensor: tensor([[   2,    0],
        [   2,    1],
        [   2,    2],
        ...,
        [   2, 9721],
        [   2, 9722],
        [   2, 9723]])
Recommended Movies for User 3: [4509, 6322, 4564, 6436, 5265, 407, 354, 7381, 3113, 4649]
